In [1]:
from math import *
import numpy as np
import pandas as pd
import datetime as dt
from datetime import timedelta
from pytz import timezone
import json

import skyfield
from skyfield.api import load
from skyfield.api import N, W, S, E, wgs84
from skyfield.api import Star
from skyfield.data import hipparcos
from skyfield.almanac import find_discrete, risings_and_settings
from skyfield.magnitudelib import planetary_magnitude
from skyfield import almanac

In [2]:
ts = load.timescale()
eph = load('de421.bsp')

sun = eph['SUN']
earth = eph['EARTH']
moon = eph['MOON']

with load.open(hipparcos.URL) as f:
    stars_df = hipparcos.load_dataframe(f)

planet_dictionary = {"Venus":"VENUS", "Mars":"MARS", "Jupiter":"JUPITER BARYCENTER", "Saturn":"SATURN BARYCENTER"}
    
star_dictionary = {"Alpheratz":677, "Ankaa":2081, "Schedar":3179, "Diphda":3419, "Achernar":7588, "Hamal":9884, "Polaris":11767, "Acamar":13847, "Menkar":14135, "Mirfak":15863, "Aldebaran":21421, "Rigel":24436, "Capella":24608, "Bellatrix":25336, "Elnath":25428, "Alnilam":26311, "Betelgeuse":27989, "Canopus":30438, "Sirius":32349, "Adhara":33579, "Procyon":37279, "Pollux":37826, "Avior":41037, "Suhail":44816, "Miaplacidus":45238, "Alphard":46390, "Regulus":49669, "Dubhe":54061, "Denebola":57632, "Gienah":59803, "Acrux":60718, "Gacrux":61084, "Alioth":62956, "Spica":65474, "Alkaid":67301, "Hadar":68702, "Menkent":68933, "Arcturus":69673, "Rigil Kent.":71683, "Kochab":72607, "Zuben'ubi":72622, "Alphecca":76267, "Antares":80763, "Atria":82273, "Sabik":84012, "Shaula":85927, "Rasalhague":86032, "Eltanin":87833, "Kaus Aust.":90185, "Vega":91262, "Nunki":92855, "Altair":97649, "Peacock":100751, "Deneb":102098, "Enif":107315, "Al Na'ir":109268, "Fomalhaut":113368, "Scheat":113881, "Markab":113963}

In [14]:
def toDegreesAndMinutes(angle):
    sign = 1
    if (angle < 0): sign = -1
    angle = angle*sign
    degrees = floor(angle)
    minutes = normal_round((angle - degrees) * 60.0, 2)
    
    return degrees*sign, minutes


def toHoursMinutesSeconds(value):
    sign = 1
    if (value < 0): sign = -1
    value = abs(value)
    hours = floor(value)
    value = (value - hours)*60.0
    minutes = floor(value)
    seconds = (value - minutes)*60.0
    return sign, hours, minutes, seconds

def fromHoursMinutesSeconds(hours, minutes, seconds):
    value = hours * 3600.0
    value += minutes*60.0
    value += seconds
    return value

def calculateJulianDate(year, month, day):
    if (month <= 2):
        month+=12
        year-=1
        
    a = int(year/100.0)
    b = 2 - a + int(a/4)
        
    return int(365.25*(year+4716)) + int(30.6001*(month+1)) + day + b - 1524.5

def calculateGMST(year, month, day, hours, minutes, seconds):
    jd = calculateJulianDate(year, month, day)

    t = (jd - 2451545.0) / 36525
    
    gmst_seconds = fromHoursMinutesSeconds(6, 41, 50.54841) 
    
    gmst_seconds += 8640184.812866*t + 0.093104*pow(t, 2) + 0.0000062*pow(t, 3) 
    
    gmst_seconds += fromHoursMinutesSeconds(hours, minutes, seconds) * 1.00273790935
    
    while (gmst_seconds < 0):
        gmst_seconds += 86400
      
    while (gmst_seconds >= 86400):
        gmst_seconds -= 86400
    
    return gmst_seconds / 3600.0


In [15]:
gmst = calculateGMST(1987, 4, 10, 19, 21, 0)
toHoursMinutesSeconds(gmst)

(1, 8, 34, 57.089592405710334)